In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sqlite3

url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
df = pd.read_parquet(url, engine='pyarrow')
df = df.head(100000)  # Limit to 100k rows

df.columns = [col.lower() for col in df.columns]

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['hour'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_week'] = df['tpep_pickup_datetime'].dt.day_name()

conn = sqlite3.connect(":memory:")
df.to_sql("trips", conn, index=False, if_exists="replace")

with PdfPages('NYC Taxi Data Analysis Report.pdf') as pdf:

    # Visual 1: Trips Per Hour
    hourly_counts = df['hour'].value_counts().sort_index()
    plt.figure(figsize=(10,5))
    hourly_counts.plot(kind='bar', color='skyblue')
    plt.title("Number of Trips by Hour of Day")
    plt.xlabel("Hour")
    plt.ylabel("Trips")
    plt.grid(True)
    plt.tight_layout()
    pdf.savefig()   
    plt.close()

    # Visual 2: Average Fare by Passenger Count
    query1 = """
    SELECT passenger_count, ROUND(AVG(total_amount), 2) as avg_fare
    FROM trips
    WHERE passenger_count > 0 AND total_amount > 0
    GROUP BY passenger_count
    ORDER BY passenger_count;
    """
    fare_by_passenger = pd.read_sql(query1, conn)

    plt.figure(figsize=(8,5))
    plt.bar(fare_by_passenger['passenger_count'], fare_by_passenger['avg_fare'], color='orange')
    plt.title("Average Fare by Passenger Count")
    plt.xlabel("Passenger Count")
    plt.ylabel("Average Fare ($)")
    plt.grid(True)
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    # Visual 3: Payment Type Distribution
    query2 = """
    SELECT payment_type, COUNT(*) as trip_count
    FROM trips
    GROUP BY payment_type
    ORDER BY trip_count DESC;
    """
    payment_dist = pd.read_sql(query2, conn)

    plt.figure(figsize=(6,6))
    plt.pie(payment_dist['trip_count'], labels=payment_dist['payment_type'], autopct='%1.1f%%',
            colors=['#66b3ff','#99ff99','#ffcc99','#ff9999'])
    plt.title("Distribution of Payment Types")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    # Visual 4: Trip Distance Histogram
    plt.figure(figsize=(10,5))
    df[df['trip_distance'] < 30]['trip_distance'].plot(kind='hist', bins=30, color='purple', edgecolor='black')
    plt.title("Trip Distance Distribution (Under 30 miles)")
    plt.xlabel("Trip Distance (miles)")
    plt.ylabel("Number of Trips")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    # Visual 5: Average Total Amount by Day of Week
    avg_fare_by_day = df.groupby("day_of_week")["total_amount"].mean().reindex(
        ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

    plt.figure(figsize=(10,5))
    avg_fare_by_day.plot(kind='bar', color='green')
    plt.title("Average Total Fare by Day of the Week")
    plt.xlabel("Day of Week")
    plt.ylabel("Average Total Amount ($)")
    plt.grid(True)
    plt.tight_layout()
    pdf.savefig()
    plt.close()

conn.close()
